In [ ]:
import pandas as pd
import numpy as np

In [2]:
my_xtrain = pd.read_parquet("dataset_train_small.parquet")
my_ytrain = pd.read_parquet("target_train_small.parquet")

In [63]:
def get_tryal(x):
    myx = x.value_counts()
    return list(({k:myx[k] if k in myx else 0 for k in range(1, 4)} ).values())

[10, 3, 0, 0, 0, 0, 0]

In [54]:
get_tryal(my_xtrain[my_xtrain['app_id'] == 6]['operation_type'])

[20, 2, 1]

In [62]:
my_xtrain['ecommerce_flag'].value_counts()

ecommerce_flag
1    28859060
2     3776915
3          12
Name: count, dtype: int64

In [3]:
appds = [1, 6, 7, 805132]

In [4]:
tryal = my_xtrain[my_xtrain['app_id'].isin(appds)]

In [24]:
new_tryal = tryal.groupby("app_id").agg({"amnt": ['mean', 'max'], "operation_kind": get_tryal})

In [71]:
new_tryal.DataFrame(new_tryal['operation_kind']['get_tryal'].to_list(), columns=["Type_1", "Type_2", "Type_3"])

In [65]:
def num_values1(x):
    if 1 in x.value_counts():
        return x.value_counts()[1]
    return 0
def num_values2(x):
    if 2 in x.value_counts():
        return x.value_counts()[2]
    return 0
def num_values3(x):
    if 3 in x.value_counts():
        return x.value_counts()[3]
    return 0

In [67]:
new_tryal2 = my_xtrain.groupby("app_id").agg({"amnt": ['mean', 'max'], "operation_kind": [num_values1, num_values2, num_values3]})

In [27]:
new_tryal[["type_1", "type_2", "type_3", "type_4", "type_5", "type_6", "type_7"]] =new_tryal['operation_kind']['get_tryal'].to_list()

In [72]:
print(my_xtrain.currency.value_counts())
print(my_xtrain.operation_type_group.value_counts())
print(my_xtrain.operation_kind.value_counts())
print(my_xtrain.ecommerce_flag.value_counts())
print(my_xtrain.income_flag.value_counts())

currency
1     31798739
2       366503
3       196557
11      134739
4        38578
6        25208
5        22847
7        17964
8        13951
10       11976
9         8925
Name: count, dtype: int64
operation_type_group
1    30425740
2     2179879
3       30364
4           4
Name: count, dtype: int64
operation_kind
1    26249909
2     2915756
3     1854686
4     1343047
5      133319
6      125354
7       13916
Name: count, dtype: int64
ecommerce_flag
1    28859060
2     3776915
3          12
Name: count, dtype: int64
income_flag
1    30236717
2     2399226
3          44
Name: count, dtype: int64
card_type
1      2338478
2      2185080
5      1763435
6      1154369
3      1008494
        ...   
173       2615
114       1310
40        1302
117        934
125         47
Name: count, Length: 169, dtype: int64


In [162]:
def num_unique_values(x):
    return len(x.unique())

def get_currency(x):
    my_uniq = x.value_counts()
    return list(({k:my_uniq[k] if k in my_uniq else 0 for k in range(1, 12)} ).values())

def get_operation_type_group(x):
    my_uniq = x.value_counts()
    return list(({k:my_uniq[k] if k in my_uniq else 0 for k in range(1, 5)} ).values())

def get_operation_kind(x):
    my_uniq = x.value_counts()
    return list(({k:my_uniq[k] if k in my_uniq else 0 for k in range(1, 8)} ).values())

def get_income_flag(x):
    my_uniq = x.value_counts()
    return list(({k:my_uniq[k] if k in my_uniq else 0 for k in range(1, 4)} ).values())

def get_payment_system(x):
    my_uniq = x.value_counts()
    return list(({k:my_uniq[k] if k in my_uniq else 0 for k in range(1, 8)} ).values())

def get_ecommerce_flag(x):
    my_uniq = x.value_counts()
    return list(({k:my_uniq[k] if k in my_uniq else 0 for k in range(1, 4)} ).values())

agg_features = {
        "transaction_number": "max",
        "amnt": ["min", "max", "mean", "median"],
        "currency": num_unique_values,
        'operation_type_group': [get_operation_type_group, "median"],
        "operation_kind": [get_operation_kind, "median"],
        'income_flag' : [get_income_flag, "median"],
        "payment_system": [get_payment_system, "median"],
        "ecommerce_flag": [get_ecommerce_flag, "mean"],
        'days_before': ['min', 'mean', 'max'],
        "country": [num_unique_values, "median"],
        'city': [num_unique_values, "median"],
        'day_of_week': [num_unique_values, "mean"],

}
features_2 = []
features_3 = []
#currency_columns = ["currency_"+str(x) for x in list(my_xtrain.currency.value_counts().index)]
operation_type_group_columns = ["operation_type_group"+str(x) for x in list(my_xtrain.operation_type_group.value_counts().index)]
operation_kind_columns = ["operation_kind"+str(x) for x in list(my_xtrain.operation_kind.value_counts().index)]
income_flag_columns = ["income_flag"+str(x) for x in list(my_xtrain.income_flag.value_counts().index)]
payment_system_columns = sorted(["payment_system"+str(x) for x in list(my_xtrain.payment_system.value_counts().index)])
ecommerce_flag_columns = ["ecommerce_flag"+str(x) for x in list(my_xtrain.ecommerce_flag.value_counts().index)]
todelete_columns = ["operation_type_group_get_operation_type_group", "operation_kind_get_operation_kind", "income_flag_get_income_flag", "payment_system_get_payment_system", "ecommerce_flag_get_ecommerce_flag"]

In [171]:
def get_new_features(dataset, target):
    features = dataset.groupby("app_id", as_index=False).agg(agg_features)
    print(features.columns)
    global features_2, features_3
    features_2 = features
    #features[currency_columns] = dataset["operation_type_group"]["get_operation_type_group"]
    features[operation_type_group_columns] = features["operation_type_group"]["get_operation_type_group"].to_list()
    features[operation_kind_columns] = features["operation_kind"]["get_operation_kind"].to_list()
    features[income_flag_columns] = features["income_flag"]["get_income_flag"].to_list()
    features[payment_system_columns] = features["payment_system"]["get_payment_system"].to_list()
    features[ecommerce_flag_columns] = features["ecommerce_flag"]["get_ecommerce_flag"].to_list()
    features_3 = features
    print(features.columns)
    features.columns = ['_'.join(col).strip('_') for col in features.columns.values]
    features = features.drop(todelete_columns, axis=1)
    features = features.join(target.set_index("app_id"), "app_id")
    return features.drop(columns=["app_id", "flag"]), features[["app_id", "flag"]]

In [175]:
new_trainx, new_trainy = get_new_features(my_xtrain, my_ytrain)

MultiIndex([(              'app_id',                         ''),
            (  'transaction_number',                      'max'),
            (                'amnt',                      'min'),
            (                'amnt',                      'max'),
            (                'amnt',                     'mean'),
            (                'amnt',                   'median'),
            (            'currency',        'num_unique_values'),
            ('operation_type_group', 'get_operation_type_group'),
            ('operation_type_group',                   'median'),
            (      'operation_kind',       'get_operation_kind'),
            (      'operation_kind',                   'median'),
            (         'income_flag',          'get_income_flag'),
            (         'income_flag',                   'median'),
            (      'payment_system',       'get_payment_system'),
            (      'payment_system',                   'median'),
          

In [178]:
new_trainx.to_csv("E:\Code works\Work\Python\Works\max_flask_app\\alfa\my_big_featurex.csv")
new_trainy.to_csv("E:\Code works\Work\Python\Works\max_flask_app\\alfa\my_big_featurey.csv")

In [184]:
my_xtest = pd.read_parquet("dataset_test.parquet")
my_ytest = pd.read_parquet("target_test_contest.parquet")

In [185]:
my_ytest['flag'] = 0

In [186]:
new_testx, new_testy = get_new_features(my_xtest, my_ytest)

MultiIndex([(              'app_id',                         ''),
            (  'transaction_number',                      'max'),
            (                'amnt',                      'min'),
            (                'amnt',                      'max'),
            (                'amnt',                     'mean'),
            (                'amnt',                   'median'),
            (            'currency',        'num_unique_values'),
            ('operation_type_group', 'get_operation_type_group'),
            ('operation_type_group',                   'median'),
            (      'operation_kind',       'get_operation_kind'),
            (      'operation_kind',                   'median'),
            (         'income_flag',          'get_income_flag'),
            (         'income_flag',                   'median'),
            (      'payment_system',       'get_payment_system'),
            (      'payment_system',                   'median'),
          

In [189]:
new_testy

,app_id,flag
0,805133,0
1,805134,0
2,805135,0
3,805136,0
4,805137,0
...,...,...
188669,1003045,0
188670,1003047,0
188671,1003048,0
188672,1003049,0


In [190]:
new_testx.to_csv("E:\Code works\Work\Python\Works\max_flask_app\\alfa\my_big_testx.csv")
new_testy.to_csv("E:\Code works\Work\Python\Works\max_flask_app\\alfa\my_big_testy.csv", index=False)

In [193]:
new_trainx.columns

Index(['transaction_number_max', 'amnt_min', 'amnt_max', 'amnt_mean',
       'amnt_median', 'currency_num_unique_values',
       'operation_type_group_median', 'operation_kind_median',
       'income_flag_median', 'payment_system_median', 'ecommerce_flag_mean',
       'days_before_min', 'days_before_mean', 'days_before_max',
       'country_num_unique_values', 'country_median', 'city_num_unique_values',
       'city_median', 'day_of_week_num_unique_values', 'day_of_week_mean',
       'operation_type_group1', 'operation_type_group2',
       'operation_type_group3', 'operation_type_group4', 'operation_kind1',
       'operation_kind2', 'operation_kind3', 'operation_kind4',
       'operation_kind5', 'operation_kind6', 'operation_kind7', 'income_flag1',
       'income_flag2', 'income_flag3', 'payment_system1', 'payment_system2',
       'payment_system3', 'payment_system4', 'payment_system5',
       'payment_system6', 'payment_system7', 'ecommerce_flag1',
       'ecommerce_flag2', 'ecommerce